In [1]:
import keypoint_moseq as kpms
import numpy as np
import os
import cv2
from itertools import groupby



project_dir = '/Users/yinazhou/DeepLabCut/examples/DeepLabCut/examples/11_1_identity_true'
model_name = '2025_11_16-22_50_50' 
video_directory = '/Users/yinazhou/DeepLabCut/examples/11_1_identity_true/videos'
trajectory_path = '/Users/yinazhou/DeepLabCut/examples/DeepLabCut/examples/11_1_identity_true/2025_11_16-22_50_50/trajectory_plots'

TARGET_SYLLABLE = 8
FPS = 30                  
BUFFER_SEC = 2           
OUTPUT_VIDEO_PATH = 'syllable_dashboard_output.mp4' 

def get_trajectory_image(syll_id, folder_path, target_height):
    """
    Loads the PNG (or PDF) for the syllable and resizes it to match video height.
    """
    png_filename = f'Syllable{syll_id}-1.png'
    png_path = os.path.join(folder_path, png_filename)
    image = None
    
    if os.path.exists(png_path):
        image = cv2.imread(png_path)
    else:
        fallback_path = os.path.join(folder_path, f'Syllable{syll_id}.png')
        if os.path.exists(fallback_path):
            image = cv2.imread(fallback_path)
    
    if image is None and PDF_SUPPORT:
        pdf_path = os.path.join(folder_path, f'Syllable{syll_id}.pdf')
        if os.path.exists(pdf_path):
            try:
                pages = convert_from_path(pdf_path, dpi=200)
                pil_image = pages[0].convert('RGB') 
                open_cv_image = np.array(pil_image) 
                image = open_cv_image[:, :, ::-1].copy() 
            except Exception as e:
                print(f"Error reading PDF: {e}")

    # placeodler if no image found
    if image is None:
        print(f"Trajectory image not found (checked {png_filename}). Creating placeholder.")
        image = np.zeros((target_height, 400, 3), dtype=np.uint8)
        cv2.putText(image, "No Trajectory Plot", (50, target_height//2), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

    # resize
    h, w, _ = image.shape
    aspect_ratio = w / h
    new_w = int(target_height * aspect_ratio)
    resized_image = cv2.resize(image, (new_w, target_height))
    
    return resized_image

def find_video_file(recording_key, video_dir):
    clean_name = recording_key.replace('_mouse1', '').replace('_mouse2', '')
    video_path = os.path.join(video_dir, clean_name + '.mp4')
    if os.path.exists(video_path): return video_path
    
    prefix = clean_name.split('_')[0]
    for file in os.listdir(video_dir):
        if file.startswith(prefix) and file.endswith('.mp4'):
            return os.path.join(video_dir, file)
    return None

def play_dashboard_clip(video_path, start_f, end_f, fps, buffer_s, mouse_id, traj_img, save_path=None):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened(): return

    # frame buffer
    buffer_frames = int(buffer_s * fps)
    play_start = max(0, start_f - buffer_frames)
    play_end = end_f + buffer_frames
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, play_start)
    
    # mouse color
    if "mouse1" in mouse_id.lower():
        color = (255, 255, 0) 
        label_text = "Mouse 1"
    else:
        color = (255, 0, 255) 
        label_text = "Mouse 2"

    print(f"Playing {mouse_id}...")

    # Scaling
    VIDEO_SCALE = 1.5   # Scale video up by 1.5x
    IMAGE_RATIO = 0.6   # Image height is 60% of the scaled video height

    writer = None # 

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break
        
        curr_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        if curr_frame > play_end: break
        
        # scale video up
        frame = cv2.resize(frame, None, fx=VIDEO_SCALE, fy=VIDEO_SCALE, interpolation=cv2.INTER_LINEAR)
        vid_h, vid_w, _ = frame.shape

        # trajecotry image make smaller
        target_img_h = int(vid_h * IMAGE_RATIO)
        
        # calculate new aspect ratio but maintain width
        img_h, img_w, _ = traj_img.shape
        scale_factor = target_img_h / img_h
        target_img_w = int(img_w * scale_factor)
        
        resized_traj = cv2.resize(traj_img, (target_img_w, target_img_h))
        
        # create a black container matching the full video height
        img_container = np.zeros((vid_h, target_img_w, 3), dtype=np.uint8)
        
        # resized image into the center of the container
        y_offset = (vid_h - target_img_h) // 2
        img_container[y_offset:y_offset+target_img_h, 0:target_img_w] = resized_traj

        # video annotation
        # Only show box if within the specific syllable start/end
        if start_f <= curr_frame <= end_f:
            # Small box in top-left corner
            cv2.rectangle(frame, (10, 10), (250, 60), color, -1) # Filled box
            cv2.putText(frame, f"{label_text}: Syl {TARGET_SYLLABLE}", (20, 45), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
            
            cv2.rectangle(frame, (0,0), (vid_w-1, vid_h-1), color, 4)

        else:
            # context mode
            cv2.putText(frame, "Waiting...", (20, 45), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200, 200, 200), 2)

        # time code
        cv2.putText(frame, f"{curr_frame/fps:.2f}s", (10, vid_h - 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

        composite = np.hstack((frame, img_container))

        # save 
        if save_path:
            if writer is None:
                h, w, _ = composite.shape
                fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
                writer = cv2.VideoWriter(save_path, fourcc, fps, (w, h))
            
            writer.write(composite)

        cv2.imshow('Syllable Dashboard', composite)
        
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break
            
    if writer:
        writer.release()
        print(f"\nSaved video to: {os.path.abspath(save_path)}")

    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

#it's execution time!
print(f"Loading results...")
results = kpms.load_results(project_dir, model_name)

found = False

for key, data in results.items():
    syllables = data['syllable']
    
    curr_idx = 0
    for syll_id, group in groupby(syllables):
        duration = len(list(group))
        
        if syll_id == TARGET_SYLLABLE:
            start_frame = curr_idx
            end_frame = curr_idx + duration
            
            vid_path = find_video_file(key, video_directory)
            
            if vid_path:
                print(f"Instance found in {key}")
                
                # determine mouse id from key
                mouse_id = "Mouse 1" if "_mouse1" in key else "Mouse 2"
                
                # oepn video to get dimensions
                cap_temp = cv2.VideoCapture(vid_path)
                ret, temp_frame = cap_temp.read()
                vid_height = temp_frame.shape[0]
                cap_temp.release()
                
                # load trjaectory plot
                traj_img = get_trajectory_image(TARGET_SYLLABLE, trajectory_path, vid_height)
                
                # play
                play_dashboard_clip(
                    vid_path, start_frame, end_frame, FPS, BUFFER_SEC, 
                    mouse_id, traj_img, save_path=OUTPUT_VIDEO_PATH
                )
                found = True
                break 
        
        curr_idx += duration
    if found: break

if not found: print("No instances found.")

Loading results...
Instance found in Test 10_WT2-WT3_cage1_TestDLC_Resnet50_10_19_familiarOct19shuffle1_snapshot_200_el_mouse1


NameError: name 'PDF_SUPPORT' is not defined